In [3]:
from app.gen_planner.python.src import territory_splitter
import geopandas as gpd

zone  = gpd.read_file('../scenarios_data/project Шлиссельбург.geojson')
zone_roads = gpd.read_file('../scenarios_data/roads_shlis.geojson')

zone_roads.geometry = zone_roads.to_crs(32636).geometry.buffer(5)

zone_splitted = territory_splitter(zone, zone_roads)
zone_splitted.explore()

In [ ]:
from app.gen_planner.python.src import GenPlanner

zones_ratio_dict = {'right_lower': 0.225, 'right_upper': 0.225, 'left_upper': 0.225, 'left_lower': 0.225,
                    'middle_lower': 0.1, 'middle_upper': 0.1, 'free_zone1': 0.1, 'free_zone2': 0.1}
fixed_zones = gpd.GeoDataFrame()
middle_lower = Point(31.015, 59.911)
middle_upper = Point(31.022, 59.922)
right_upper = Point(31.046, 59.922)
right_lower = Point(31.043, 59.911)
left_upper = Point(30.999, 59.927)
left_lower = Point(30.993, 59.912)
fixed_2zones = gpd.GeoDataFrame(
    {'zone': ['middle_lower', 'middle_upper']},
    geometry=[middle_lower, middle_upper],
    crs=4326)

fixed_1zone2p = gpd.GeoDataFrame(
    {'zone': ['middle_lower', 'middle_lower']},
    geometry=[middle_lower, middle_upper],
    crs=4326)

fixed_6zones = gpd.GeoDataFrame(
    {'zone': ['middle_lower', 'middle_upper', 'right_upper', 'right_lower', 'left_upper', 'left_lower']},
    geometry=[middle_lower, middle_upper, right_upper, right_lower, left_upper, left_lower],
    crs=4326)

fixed_zones = fixed_6zones

generated_zones, generated_roads = GenPlanner(territory, dev_mod=True).split_features(zones_ratio_dict=zones_ratio_dict,
                                                                                      fixed_zones=fixed_zones)